In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!unzip open.zip

Archive:  open.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# LabelEncoder로 SUBCLASS를 숫자로 변환
label_encoder = LabelEncoder()
train_df['SUBCLASS'] = label_encoder.fit_transform(train_df['SUBCLASS'])

# 학습된 클래스 확인
print("LabelEncoder 클래스 매핑 (정수 -> 암종):")
for idx, class_label in enumerate(label_encoder.classes_):
    print(f"{idx} -> {class_label}")

# 모든 값이 'WT'인 열을 제거하는 작업
columns_to_drop = [col for col in train_df.columns if (train_df[col] == 'WT').all()]

# 열 제거
train_df_cleaned = train_df.drop(columns=columns_to_drop)
test_df_cleaned = test_df.drop(columns=columns_to_drop)

# X, y 분리
X = train_df_cleaned.drop(columns=['ID', 'SUBCLASS'])
y = train_df_cleaned['SUBCLASS']
X_test = test_df_cleaned.drop(columns=['ID'])

# 범주형 변이 정보를 LabelEncoder로 처리 (학습 데이터만 인코딩)
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])  # 학습 데이터 인코딩

        # 학습 데이터에 있는 값만 인코딩, 테스트 데이터는 그대로 두기
        le_classes = set(le.classes_)
        X_test[col] = X_test[col].map(lambda x: le.transform([x])[0] if x in le_classes else -1)  # 학습 데이터에 없는 값은 -1로 처리

# 학습 및 검증 데이터 분리 (stratify=y로 클래스 균형 유지)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# RandomForest 모델 생성 및 학습
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# 검증 데이터에 대한 예측
y_pred_val = rf_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print(f'Random Forest 검증 데이터 정확도: {val_accuracy}')

# 테스트 데이터 예측 (학습 데이터에 없는 변이 값은 -1로 처리됨)
y_pred_test = rf_clf.predict(X_test)

# LabelEncoder로 학습된 클래스 확인
class_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print(f"클래스 매핑: {class_mapping}")

# 예측된 정수형 SUBCLASS 값을 원래 암종 이름으로 수동 변환
y_pred_test_labels = [class_mapping[pred] for pred in y_pred_test]

# 제출 파일 생성 (테스트 데이터에서 학습 데이터에 없는 변이 값은 그대로 유지)
submission_df = pd.DataFrame({
    'ID': test_df_cleaned['ID'],
    'SUBCLASS': y_pred_test_labels  # 원래 SUBCLASS 이름으로 출력
})

# 제출 파일 생성
submission_df.to_csv('submission_random_forest_fixed2.csv', index=False)


LabelEncoder 클래스 매핑 (정수 -> 암종):
0 -> ACC
1 -> BLCA
2 -> BRCA
3 -> CESC
4 -> COAD
5 -> DLBC
6 -> GBMLGG
7 -> HNSC
8 -> KIPAN
9 -> KIRC
10 -> LAML
11 -> LGG
12 -> LIHC
13 -> LUAD
14 -> LUSC
15 -> OV
16 -> PAAD
17 -> PCPG
18 -> PRAD
19 -> SARC
20 -> SKCM
21 -> STES
22 -> TGCT
23 -> THCA
24 -> THYM
25 -> UCEC
Random Forest 검증 데이터 정확도: 0.2884770346494762
클래스 매핑: {0: 'ACC', 1: 'BLCA', 2: 'BRCA', 3: 'CESC', 4: 'COAD', 5: 'DLBC', 6: 'GBMLGG', 7: 'HNSC', 8: 'KIPAN', 9: 'KIRC', 10: 'LAML', 11: 'LGG', 12: 'LIHC', 13: 'LUAD', 14: 'LUSC', 15: 'OV', 16: 'PAAD', 17: 'PCPG', 18: 'PRAD', 19: 'SARC', 20: 'SKCM', 21: 'STES', 22: 'TGCT', 23: 'THCA', 24: 'THYM', 25: 'UCEC'}
